In [1]:
import numpy as np

In [ ]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create grid of 2D points
# x = np.linspace(2e-7, 1e-5, 20)  # specify bounds for x
# y = np.linspace(2e-7, 1e-5, 20)  # specify bounds for y
x = np.linspace(2e5, 1.5e8, 20)  # specify bounds for x
y = np.linspace(600, 7500, 20)  # specify bounds for y
X, Y = np.meshgrid(x, y)
points = np.vstack([X.ravel(), Y.ravel()]).T

bounds =[40e6, 56e6]

# Apply logarithm to each dimension
#log_points = np.array([np.log(points[:, 0]), points[:, 1]]).reshape(-1, 2)
X, Y = np.meshgrid(np.log(x), y)
log_points = np.vstack([X.ravel(), Y.ravel()]).T
log_bounds = [np.log(bounds[0]), np.log(bounds[1])]

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("Original Points", "Log-Transformed Points"))

# Add original points subplot
fig.add_trace(go.Scatter(x=points[:, 0], y=points[:, 1], mode='markers', name='Original Points'), row=1, col=1)

fig.add_shape(type="line", x0=bounds[0], y0=600, x1=bounds[0], y1=7500, line=dict(color="red", width=3), row=1, col=1)
fig.add_shape(type="line", x0=bounds[1], y0=600, x1=bounds[1], y1=7500, line=dict(color="red", width=3), row=1, col=1)

# Add log-transformed points subplot
fig.add_trace(go.Scatter(x=log_points[:, 0], y=log_points[:, 1], mode='markers', name='Log-Transformed Points'), row=1, col=2)

fig.add_shape(type="line", x0=log_bounds[0], y0=600, x1=log_bounds[0], y1=7500, line=dict(color="blue", width=3), row=1, col=2)
fig.add_shape(type="line", x0=log_bounds[1], y0=600, x1=log_bounds[1], y1=7500, line=dict(color="blue", width=3), row=1, col=2)

# Update layout
fig.update_layout(height=600, width=1200, title_text="Original and Log-Transformed Points")
#fig.update_xaxes(constrain='domain')  
#fig.update_yaxes(scaleanchor= 'x')
# Show plot
fig.show()


In [5]:
import numpy as np
import warnings

In [6]:
# Load kedro scaler
p = catalog.load('parameters')

features=p["features"]
targets=p["targets"]
additional_values = p["additional_values"]
names=p["names"]

scaler=catalog.load('scaler')
treatment=catalog.load('treatment')

[03/13/24 22:27:38] INFO     Loading data from 'parameters' (MemoryDataSet)...                  ]8;id=328833;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=74553;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'scaler' (PickleDataSet)...                      ]8;id=524812;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=776559;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Loading data from 'treatment' (PickleDataSet)...                   ]8;id=10693;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=153495;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [20]:
from pyDOE import lhs

# Number of samples
num_samples = 100

# Get bounds from treatment
bounds = np.array([treatment.variables_ranges[f]['bounds'] for f in features])

# Generate Latin Hypercube Samples
samples = lhs(n=3, samples=num_samples)



In [21]:
warnings.warn("Using LHS for sampling! Sampling points of dim {}".format(samples.shape))

# Scale samples to the specified bounds for each dimension
#samples = np.zeros(samples.shape)
new_samples = np.zeros(samples.shape)
for i in range(len(bounds)):
    new_samples[:, i] = bounds[i, 0] + samples[:, i] * (bounds[i, 1] - bounds[i, 0])

# Scale samples down again using the treatment scaler (some variables may be log-scaled)
new_samples = treatment.scaler.transform_features(new_samples)

[03/13/24 22:31:43] WARNING  /tmp/ipykernel_169076/2200028520.py:1: UserWarning:                    ]8;id=643263;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=316179;file:///home/jack/miniconda3/envs/sampler/lib/python3.10/warnings.py#109\109]8;;\
                                                                                                                   
                             Using LHS for sampling! Sampling points of dim (100, 3)                               
                                                                                                                   
                                                                                                                   

In [27]:
# plot 2d samples using plotly
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(go.Scatter(x=samples[:, 0], y=samples[:, 2], mode='markers'))
fig.add_trace(go.Scatter(x=new_samples[:, 0], y=new_samples[:, 2], mode='markers', marker=dict(color='red')))
fig.show()


In [50]:
import pandas as pd

path = "../data/05_model_input/100_linear_LHS.csv"
df = pd.read_csv(path)
df = df[features]
df

,r_ext_pAl,r_ext_pMeO,pAl_richness
0,0.030924,0.133746,0.504685
1,0.189274,0.735465,0.381894
2,0.310786,0.238767,0.162763
3,0.626804,0.673606,0.948136
4,0.496128,0.538352,0.898604
...,...,...,...
95,0.716376,0.162881,0.290646
96,0.087966,0.970158,0.417442
97,0.158067,0.436741,0.208169
98,0.805028,0.091185,0.535617


In [43]:
# plot 2d samples using plotly
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.values[:, 0], y=df.values[:, 2], mode='markers', marker=dict(color='red')))
fig.show()

In [55]:
path = "../data/02_intermediate/100_linear_LHS_physical.csv"
df_physical = pd.read_csv(path)
df_physical = df_physical[features]
df_physical

,r_ext_pAl,r_ext_pMeO,pAl_richness
0,5.030505e-07,1.510715e-06,2.514056
1,2.054883e-06,7.407555e-06,2.145682
2,3.245706e-06,2.539921e-06,1.488289
3,6.342679e-06,6.801342e-06,3.844408
4,5.062054e-06,5.475850e-06,3.695811
...,...,...,...
95,7.220489e-06,1.796238e-06,1.871938
96,1.062069e-06,9.707552e-06,2.252325
97,1.749060e-06,4.480061e-06,1.624508
98,8.089279e-06,1.093618e-06,2.606852


In [57]:
physical_rescaled = treatment.scaler.transform_features(df_physical.values)

In [59]:
# plot 2d samples using plotly
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.values[:, 0], y=df.values[:, 2], mode='markers', marker=dict(color='blue')))
fig.add_trace(go.Scatter(x=physical_rescaled[:, 0], y=physical_rescaled[:, 2], mode='markers', marker=dict(color='red')))

fig.show()